<a href="https://colab.research.google.com/github/namangarg1805/Data-Analytics/blob/main/Data_Scraping_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Goal:**
Scrape Recent Job Postings from Assigned Job Portals

In [ ]:
!pip install requests
!pip install html5lib
!pip install bs4

In [ ]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
# Job Title
# o Company Name
# o Location
# o Job Type (e.g., Full-time, Part-time, Contract)
# o Date Posted
# o Job Description (brief summary)
# o Contact Information (if available)
# o Application Link (direct URL to apply)


In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
    'Referer': 'https://www.google.com/',  # Add a referrer if needed
}

In [ ]:
def scrape_content(url):
    response = requests.get(url,headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup

def scrape_multiple_pages(base_url, num_pages):
    all_data = []
    for page_num in range(1, num_pages + 1):
        url = f"{base_url}?page={page_num}"
        soup = scrape_content(url)

        # Extract data from each page (replace with actual selectors)
        title_list = []
        for div in soup.find_all("span", attrs={"class": "wrap-title seo_title"}):
            title = div.text.strip() # Remove leading/trailing whitespace
            words_remove_index = title.find("Jobs Opening")
            new_title = title[:words_remove_index]
            title_list.append(new_title)

        company_list = []
        for div in soup.find_all("h3", attrs={"class": "company-name"}):
            company = div.text.strip()
            company_list.append(company)

        location_list = []
        for div in soup.find_all("span", attrs={"class": "job-location"}):
            location = div.text.strip()
            location_list.append(location)

        job_id_list = []
        for div in soup.find_all("div"):
            if div.has_attr("data-job-id"):
                job_id = div.get("data-job-id")
                job_id_list.append(job_id)



        job_url_list = ["-"+job_id_list[i] for i in range(len(title_list))]


        job_date_list=[]
        job_type_list = []
        for job_url in job_url_list:
            new_url = base_url+job_url
            soup = scrape_content(new_url)
            job_type = soup.find_all("span", attrs={"class": "job-part-detail"})
            if job_type==None:
                job_type="Unknown"
            else:
                job_type=job_type.text.strip()
            job_date = soup.find("span",attrs ={"class":"posted-on  padding-none"})
            if job_date==None:
                job_date = "19 jan"
            else:
                job_date=job_date.text.strip()
            job_type_list.append(job_type)
            job_date_list.append(job_date)

        # print(job_desc_url_list)
        # job_desc_list=[]
        # for desc_url in job_desc_url_list:
        #     new_url = base_url+desc_url
        #     # print(new_url)
        #     soup = scrape_content(new_url)
        #     desc = soup.find("div", attrs={"class": "job-desc"})
        #     if desc!=None:
        #         job_desc = desc.text
        #         words_remove_index = job_desc.find("More information about this")
        #         new_job_desc = job_desc[:words_remove_index]
        #         # print(new_job_desc)
        #         job_desc_list.append(new_job_desc)

        page_data = {
            "title": title_list,
            "company": company_list,
            "location": location_list,
            "job Id":job_id_list,
            "date posted":job_date_list,
            "job type":job_type_list,
            "Job Url":job_url_list
        }

        all_data.extend([{ "title":title, "company":company, "location":location, "job Id":job_id, "date posted":job_date,"job_type":job_type,"Job Url":job_url} for title, company, location, job_id, job_date,job_type,job_url in zip(title_list, company_list, location_list, job_id_list, job_date_list,job_type_list,job_url_list)])

    return all_data

base_url = "https://www.freshersworld.com/"
jobs_search = "jobs/"
url = base_url+jobs_search
num_pages_to_scrape = 1
scraped_data = scrape_multiple_pages(url, num_pages_to_scrape)
df = pd.DataFrame(scraped_data)
df.shape

(20, 7)

In [ ]:
df.head()

,title,company,location,job Id,date posted,job_type,Job Url
0,Service Technician,Vishakha Enterprises - IFB Authorized service ...,Chennai,2607624,19 jan,mechanic fitter production,-2607624
1,Field Sales Executive,Autoverse Mobility Private Limited,Bangalore,2607589,19 jan,sales marketing executive,-2607589
2,Call Centre Assistant,Asirvad Micro Finance Limited,Thrissur,2607585,19 jan,bpo telecaller,-2607585
3,Accounts And Billing Executive,Sagar Services,Mumbai,2607052,19 jan,accountant,-2607052
4,Account Executive,Career Job Solution,Delhi,2606474,19 jan,accountant,-2606474


In [ ]:
# res = df.to_excel("Sparix G.xlsx")

In [ ]:
res = scrape_content(url)
res.text

'Fresher Jobs: Latest Jobs for Freshers In India. Freshers Govt jobs & MNC jobs vacanciesFreshersworld does not charge any amount for job placement. Beware of fraudsters who ask you to pay on the pretext of giving a job. Know MoreJobsPlacement PaperPremium MembershipCoursesCareer DirectoryTraining InstitutesTraining CoursesColleges & UniversitiesEducational ConsultantHigher Study in IndiaMBA/PGDMMCACareer TipsResume TipsInterview TipsGD Tips & MocksStudy AbroadIrelandSingaporeUKUSANew ZealandCanadaTest PreparationBank Exams / IBPSCAT / MATCivil Services - IAS / IPSDefence / SSBGATEGMATGREIELTS / TOEFLComputer Training CoursesBig Data / HadoopNetworking / CCNAC / C++C#Java /J2EEEmbedded SystemsCloud ComputingPHPLinux / UnixMySQL / DatabaseOracleSoftware TestingApp Development (Android / iOS/ Windows)Computer HardwareData WarehousingPLC/AutomationRuby on RailsSAPWeb DesigningNon-IT CoursesAutocad / CADBPO - Call centerSoft Skills /Personality Development& Many More..Jobs & Career VideosL